This module is for __OHLCV DAILY__ :

    ** 1. Merge the pumped signals with time stamps to get vertical format for voting
    ** 2. Append the output whethere the coin was pumped or not (binary 0)

In [1]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

## Import the pumped data

### Step 1. Create the timestamp for the first hours 

In [5]:
p = pd.read_pickle("../pump_data_with_social.pkl")
p = p[['channelLink','channelTitle','currency','exchange','signalTime','pumptime_edited','timestamp']]
p.head(2)

,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,Binance,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,Binance,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032


Ther are .... unique points in time...

In [6]:
#Number of unique time points
len(pd.unique(p['timestamp']))

422

In [7]:
#Number of unique coins points
len(pd.unique(p['currency']))

154

In [8]:
3600*24

86400

In [9]:
1562346032 - (1562346032 % (86400))

1562284800

..Strange beheavior, must write each function one by one...

In [10]:
def getDaystamp(dfo: pd.DataFrame, num_days: int=1, var:str='timestamp'):
    '''
    Append a column that contains the timestamp at 12:00 AM (0:00) of the
    original var variable.
    :param df: dataframe in question 
    :param num_date: number of dates to go back to
    :param var str: name of the variable that has original timestamp
    '''
    df = dfo.copy()
    df[var+'_int'] = df[var].apply(int)
    for d in range(num_days+1):
        df[var+'_d'+str(d)] = df[var+'_int'].apply(lambda t : t - (t % (3600*24)) - d*(3600*24))
    df = df.drop(columns=var+'_int')
    return df

In [11]:
num_days = 31
p = getDaystamp(p,num_days)
p.head()

,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp,timestamp_d0,timestamp_d1,timestamp_d2,timestamp_d3,timestamp_d4,timestamp_d5,timestamp_d6,timestamp_d7,timestamp_d8,timestamp_d9,timestamp_d10,timestamp_d11,timestamp_d12,timestamp_d13,timestamp_d14,timestamp_d15,timestamp_d16,timestamp_d17,timestamp_d18,timestamp_d19,timestamp_d20,timestamp_d21,timestamp_d22,timestamp_d23,timestamp_d24,timestamp_d25,timestamp_d26,timestamp_d27,timestamp_d28,timestamp_d29,timestamp_d30,timestamp_d31
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,Binance,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999,1564099200,1564012800,1563926400,1563840000,1563753600,1563667200,1563580800,1563494400,1563408000,1563321600,1563235200,1563148800,1563062400,1562976000,1562889600,1562803200,1562716800,1562630400,1562544000,1562457600,1562371200,1562284800,1562198400,1562112000,1562025600,1561939200,1561852800,1561766400,1561680000,1561593600,1561507200,1561420800
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,Binance,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032,1562284800,1562198400,1562112000,1562025600,1561939200,1561852800,1561766400,1561680000,1561593600,1561507200,1561420800,1561334400,1561248000,1561161600,1561075200,1560988800,1560902400,1560816000,1560729600,1560643200,1560556800,1560470400,1560384000,1560297600,1560211200,1560124800,1560038400,1559952000,1559865600,1559779200,1559692800,1559606400
2,https://t.me/MaximumPump,Maximum Pump,ADX,Binance,2019-04-25T19:15:21,2019-04-25 19:15:21+00:00,1556219721,1556150400,1556064000,1555977600,1555891200,1555804800,1555718400,1555632000,1555545600,1555459200,1555372800,1555286400,1555200000,1555113600,1555027200,1554940800,1554854400,1554768000,1554681600,1554595200,1554508800,1554422400,1554336000,1554249600,1554163200,1554076800,1553990400,1553904000,1553817600,1553731200,1553644800,1553558400,1553472000
3,https://t.me/bitcoinpumpgroup,Bitcoin Pump Group,WAN,Binance,2019-08-30T16:41:30Z,2019-08-30 16:41:30+00:00,1567183290,1567123200,1567036800,1566950400,1566864000,1566777600,1566691200,1566604800,1566518400,1566432000,1566345600,1566259200,1566172800,1566086400,1566000000,1565913600,1565827200,1565740800,1565654400,1565568000,1565481600,1565395200,1565308800,1565222400,1565136000,1565049600,1564963200,1564876800,1564790400,1564704000,1564617600,1564531200,1564444800
4,https://t.me/bitcoinprofitcoach,Bitcoin Profit Coach,QSP,Binance,2019-08-29T19:46:43Z,2019-08-29 19:46:43+00:00,1567108003,1567036800,1566950400,1566864000,1566777600,1566691200,1566604800,1566518400,1566432000,1566345600,1566259200,1566172800,1566086400,1566000000,1565913600,1565827200,1565740800,1565654400,1565568000,1565481600,1565395200,1565308800,1565222400,1565136000,1565049600,1564963200,1564876800,1564790400,1564704000,1564617600,1564531200,1564444800,1564358400


### Test an Processed Satoshi dataset before Merging

In [9]:
s = pd.read_pickle("../../Coin_Data/Daily/Processed_Data/Sat_Daily_ADX_at_Bittrex.pkl")

In [10]:
s.head()

,time,close,high,low,open,volumefrom,volumeto,exchange,date,date_utc
0,1518048000,13200.0,14540.0,13190.0,13910.0,3.493146e+13,4.848000e+09,Bittrex,2018-02-07 16:00:00,2018-02-08 00:00:00+00:00
1,1518134400,14040.0,14310.0,13080.0,13200.0,3.032950e+13,4.167000e+09,Bittrex,2018-02-08 16:00:00,2018-02-09 00:00:00+00:00
2,1518220800,17340.0,17340.0,13830.0,14040.0,1.549686e+14,2.397300e+10,Bittrex,2018-02-09 16:00:00,2018-02-10 00:00:00+00:00
3,1518307200,15240.0,17480.0,14700.0,17340.0,9.172312e+13,1.448300e+10,Bittrex,2018-02-10 16:00:00,2018-02-11 00:00:00+00:00
4,1518393600,14630.0,15350.0,14600.0,15240.0,2.714896e+13,4.057000e+09,Bittrex,2018-02-11 16:00:00,2018-02-12 00:00:00+00:00


In [11]:
fin_vars = ['open','high','low','close','volumefrom','volumeto','date','date_utc']

### Step 2: Loop throug the processed social data file and merge based on timestemp
... We do not need to join by the coinID nor coin name. 

Just the __timestamps__ and __EXCHANGE__ are good enough. 
Since we want to retrieve the values for all available timestamps.

## DAILY

In [12]:
dir_prefix = "../../Coin_Data/Daily/Processed_Data/"
ohlc_error = []
x_final = pd.DataFrame()
t_temp = pd.DataFrame()
for filename in os.listdir(dir_prefix):
    if 'pkl' in filename:
        _,_,coin_name,_,exchange = filename.split('_')
        exchange = exchange[:-4]
        try: 
            print("Processing file...",filename)
            l = p.copy()
            for d in range(num_days+1):
                if 'timestamp_d'+str(d) not in l.columns: 
                    print('Info. on day_{} not in the left table!'.format(d))
                    continue
                x = pd.read_pickle(dir_prefix+filename)
                x.columns = [c+'_d'+str(d) if c != 'time' else 'time' for c in x.columns]
                #Merge the data based on time and exchange
              
                l = pd.merge(left=l,left_on=['timestamp_d'+str(d),'exchange']
                             ,right=x, right_on=["time","exchange_d"+str(d)]
                             ,how='inner')
#                 print(l.shape)
                l = l.drop(columns=['time',"exchange_d"+str(d)])
            #Save the file in the processed_directory
            l['coin_name'] = coin_name
            #Append all the data on top of each other for processing; 
            x_final = x_final.append(l)
        except: 
            e = sys.exc_info()[0]
            print( "<p>Error: %s</p>" % e )
            ohlc_error.append(filename)
            continue

Processing file... Sat_Daily_TEC_at_Yobit.pkl
Processing file... Sat_Daily_BCH_at_Yobit.pkl
Processing file... Sat_Daily_NULS_at_Binance.pkl
Processing file... Sat_Daily_LOOM_at_Binance.pkl
Processing file... Sat_Daily_ENJ_at_Bittrex.pkl
Processing file... Sat_Daily_VIB_at_Binance.pkl
Processing file... Sat_Daily_ZIL_at_Bittrex.pkl
Processing file... Sat_Daily_STORJ_at_Binance.pkl
Processing file... Sat_Daily_BAT_at_Yobit.pkl
Processing file... Sat_Daily_TKN_at_Yobit.pkl
Processing file... Sat_Daily_FTM_at_Binance.pkl
Processing file... Sat_Daily_LEND_at_Binance.pkl
Processing file... Sat_Daily_INFX_at_Bittrex.pkl
Processing file... Sat_Daily_XZC_at_Binance.pkl
Processing file... Sat_Daily_SNT_at_Yobit.pkl
Processing file... Sat_Daily_ADX_at_Bittrex.pkl
Processing file... Sat_Daily_FET_at_Binance.pkl
Processing file... Sat_Daily_MET_at_Yobit.pkl
Processing file... Sat_Daily_POLY_at_Binance.pkl
Processing file... Sat_Daily_SNT_at_Bittrex.pkl
Processing file... Sat_Daily_PIVX_at_Yobit.pk

Processing file... Sat_Daily_ARK_at_Binance.pkl
Processing file... Sat_Daily_CVC_at_Binance.pkl
Processing file... Sat_Daily_CTXC_at_Bittrex.pkl
Processing file... Sat_Daily_MITH_at_Binance.pkl
Processing file... Sat_Daily_VIA_at_Bittrex.pkl
Processing file... Sat_Daily_BCH_at_Binance.pkl
Processing file... Sat_Daily_DATA_at_Binance.pkl
Processing file... Sat_Daily_GAM_at_Bittrex.pkl
Processing file... Sat_Daily_THETA_at_Binance.pkl
Processing file... Sat_Daily_GRS_at_Binance.pkl
Processing file... Sat_Daily_PIVX_at_Binance.pkl
Processing file... Sat_Daily_BNT_at_Yobit.pkl
Processing file... Sat_Daily_PPT_at_Binance.pkl
Processing file... Sat_Daily_ELF_at_Bittrex.pkl
Processing file... Sat_Daily_AE_at_Yobit.pkl
Processing file... Sat_Daily_ING_at_Yobit.pkl
Processing file... Sat_Daily_TNB_at_Binance.pkl
Processing file... Sat_Daily_UKG_at_Bittrex.pkl
Processing file... Sat_Daily_BAT_at_Binance.pkl
Processing file... Sat_Daily_SALT_at_Bittrex.pkl
Processing file... Sat_Daily_GVT_at_Yobi

In [15]:
print(ohlc_error)
print(l.shape)

[]
(47, 296)


#We might have to check whether there is a non-zero data somewhere....

In [16]:
x_final = x_final.reset_index(drop=True)
x_final = x_final.sort_values(['timestamp','exchange'])
x_final = x_final.drop_duplicates()

In [17]:
x_final.filter(regex="open")

,open_d0,open_d1,open_d2,open_d3,open_d4,open_d5,open_d6,open_d7,open_d8,open_d9,open_d10,open_d11,open_d12,open_d13,open_d14,open_d15,open_d16,open_d17,open_d18,open_d19,open_d20,open_d21,open_d22,open_d23,open_d24,open_d25,open_d26,open_d27,open_d28,open_d29,open_d30,open_d31
198,294.0,294.0,294.0,294.0,298.0,298.0,298.0,323.0,323.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,504.0,504.0,504.0,504.0,504.0,378.0,378.0,586.0,586.0,586.0,586.0,586.0,586.0
397,11040000.0,11250000.0,11680000.0,11950000.0,11400000.0,11170000.0,11200000.0,10990000.0,11100000.0,10900000.0,11010000.0,11170000.0,11380000.0,11130000.0,11340000.0,11590000.0,11770000.0,11850000.0,11620000.0,11620000.0,11660000.0,11190000.0,11640000.0,11610000.0,12190000.0,12360000.0,12420000.0,12640000.0,13040000.0,13280000.0,13420000.0,13180000.0
2050,4790.0,5124.0,5147.0,5500.0,5001.0,5100.0,4975.0,4300.0,4220.0,4100.0,4100.0,3900.0,4096.0,4200.0,3900.0,4075.0,4166.0,4200.0,4179.0,4000.0,3900.0,3700.0,3800.0,3800.0,4101.0,4150.0,4150.0,4100.0,4600.0,4100.0,3857.0,3900.0
2249,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3515,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57016,2350.0,2310.0,2380.0,2420.0,2260.0,2500.0,2200.0,2240.0,2170.0,2120.0,2140.0,2310.0,2260.0,2090.0,2100.0,2080.0,2160.0,2070.0,2160.0,2080.0,2060.0,2100.0,1860.0,2060.0,2170.0,2420.0,2730.0,2660.0,2790.0,2770.0,2780.0,2910.0
57356,6040.0,5650.0,5540.0,5330.0,5620.0,6310.0,5410.0,5590.0,5480.0,5110.0,4970.0,5250.0,5180.0,5040.0,5530.0,5510.0,5530.0,5500.0,4970.0,4940.0,4620.0,5250.0,5460.0,6030.0,6550.0,7330.0,7490.0,7580.0,7790.0,8150.0,8660.0,8830.0
57743,1985.0,1916.0,2075.0,2090.0,2080.0,2349.0,2056.0,2025.0,1993.0,1865.0,1830.0,1887.0,1884.0,1806.0,1882.0,1894.0,1965.0,1938.0,1810.0,1891.0,1751.0,1925.0,2092.0,2246.0,2163.0,2338.0,2271.0,2329.0,2331.0,2479.0,2741.0,2756.0
58083,2130.0,2160.0,2240.0,2230.0,2230.0,2420.0,2130.0,2210.0,2140.0,2070.0,2090.0,2250.0,2210.0,2090.0,2130.0,2230.0,2350.0,2240.0,2220.0,2280.0,2160.0,2320.0,2450.0,2600.0,2490.0,2770.0,2820.0,2800.0,2890.0,2650.0,2940.0,3160.0


__!!__ Notice that we have only __rougly half__ the number of obs. as the social media only case, 
but the additional constraints of exhange explains that (Few coins listed at each exchange).

*** Now we have the time series data up to 1 month in the past!
*** So now we can start preparing the data and sort the categories together?

In [19]:
print(x_final.shape)
x_final.head()

(43506, 296)


,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp,timestamp_d0,timestamp_d1,timestamp_d2,timestamp_d3,timestamp_d4,timestamp_d5,timestamp_d6,timestamp_d7,timestamp_d8,timestamp_d9,timestamp_d10,timestamp_d11,timestamp_d12,timestamp_d13,timestamp_d14,timestamp_d15,timestamp_d16,timestamp_d17,timestamp_d18,timestamp_d19,timestamp_d20,timestamp_d21,timestamp_d22,timestamp_d23,timestamp_d24,timestamp_d25,timestamp_d26,timestamp_d27,timestamp_d28,timestamp_d29,timestamp_d30,timestamp_d31,close_d0,high_d0,low_d0,open_d0,volumefrom_d0,volumeto_d0,date_d0,date_utc_d0,close_d1,high_d1,low_d1,open_d1,volumefrom_d1,volumeto_d1,date_d1,date_utc_d1,close_d2,high_d2,low_d2,open_d2,volumefrom_d2,volumeto_d2,date_d2,date_utc_d2,close_d3,high_d3,low_d3,open_d3,volumefrom_d3,volumeto_d3,date_d3,date_utc_d3,close_d4,high_d4,low_d4,open_d4,volumefrom_d4,volumeto_d4,date_d4,date_utc_d4,close_d5,high_d5,low_d5,open_d5,volumefrom_d5,volumeto_d5,date_d5,date_utc_d5,close_d6,high_d6,low_d6,open_d6,volumefrom_d6,volumeto_d6,date_d6,date_utc_d6,close_d7,high_d7,low_d7,open_d7,volumefrom_d7,volumeto_d7,date_d7,date_utc_d7,close_d8,high_d8,low_d8,open_d8,volumefrom_d8,volumeto_d8,date_d8,date_utc_d8,close_d9,high_d9,low_d9,open_d9,volumefrom_d9,volumeto_d9,date_d9,date_utc_d9,close_d10,high_d10,low_d10,open_d10,volumefrom_d10,volumeto_d10,date_d10,date_utc_d10,close_d11,high_d11,low_d11,open_d11,volumefrom_d11,volumeto_d11,date_d11,date_utc_d11,close_d12,high_d12,low_d12,open_d12,volumefrom_d12,volumeto_d12,date_d12,date_utc_d12,close_d13,high_d13,low_d13,open_d13,volumefrom_d13,volumeto_d13,date_d13,date_utc_d13,close_d14,high_d14,low_d14,open_d14,volumefrom_d14,volumeto_d14,date_d14,date_utc_d14,close_d15,high_d15,low_d15,open_d15,volumefrom_d15,volumeto_d15,date_d15,date_utc_d15,close_d16,high_d16,low_d16,open_d16,volumefrom_d16,volumeto_d16,date_d16,date_utc_d16,close_d17,high_d17,low_d17,open_d17,volumefrom_d17,volumeto_d17,date_d17,date_utc_d17,close_d18,high_d18,low_d18,open_d18,volumefrom_d18,volumeto_d18,date_d18,date_utc_d18,close_d19,high_d19,low_d19,open_d19,volumefrom_d19,volumeto_d19,date_d19,date_utc_d19,close_d20,high_d20,low_d20,open_d20,volumefrom_d20,volumeto_d20,date_d20,date_utc_d20,close_d21,high_d21,low_d21,open_d21,volumefrom_d21,volumeto_d21,date_d21,date_utc_d21,close_d22,high_d22,low_d22,open_d22,volumefrom_d22,volumeto_d22,date_d22,date_utc_d22,close_d23,high_d23,low_d23,open_d23,volumefrom_d23,volumeto_d23,date_d23,date_utc_d23,close_d24,high_d24,low_d24,open_d24,volumefrom_d24,volumeto_d24,date_d24,date_utc_d24,close_d25,high_d25,low_d25,open_d25,volumefrom_d25,volumeto_d25,date_d25,date_utc_d25,close_d26,high_d26,low_d26,open_d26,volumefrom_d26,volumeto_d26,date_d26,date_utc_d26,close_d27,high_d27,low_d27,open_d27,volumefrom_d27,volumeto_d27,date_d27,date_utc_d27,close_d28,high_d28,low_d28,open_d28,volumefrom_d28,volumeto_d28,date_d28,date_utc_d28,close_d29,high_d29,low_d29,open_d29,volumefrom_d29,volumeto_d29,date_d29,date_utc_d29,close_d30,high_d30,low_d30,open_d30,volumefrom_d30,volumeto_d30,date_d30,date_utc_d30,close_d31,high_d31,low_d31,open_d31,volumefrom_d31,volumeto_d31,date_d31,date_utc_d31,coin_name
57016,https://t.me/bitcoinpumpgroup,Bitcoin Pump Group,WAN,Binance,2019-08-30T16:41:30Z,2019-08-30 16:41:30+00:00,1567183290,1567123200,1567036800,1566950400,1566864000,1566777600,1566691200,1566604800,1566518400,1566432000,1566345600,1566259200,1566172800,1566086400,1566000000,1565913600,1565827200,1565740800,1565654400,1565568000,1565481600,1565395200,1565308800,1565222400,1565136000,1565049600,1564963200,1564876800,1564790400,1564704000,1564617600,1564531200,1564444800,2330.0,2390.0,2280.0,2350.0,4.027936e+13,9.329000e+08,2019-08-29 17:00:00,2019-08-30 00:00:00+00:00,2350.0,2450.0,2280.0,2310.0,3.505532e+13,8.267000e+08,2019-08-28 17:00:00,2019-08-29 00:00:00+00:00,2310.0,2500.0,2240.0,2380.0,6.532744e+13,1.536000e+09,2019-08-27 17:00:00,2019-08-28 00:00:00+00:00,2380.0,2530.0,2330.0,2

In [12]:
col_dict = dict()
for f in ['open','high','low','close','volumefrom','volumeto']:
    col_dict[f] = [f+"_d"+str(d) for d in range(num_days + 1)]

In [13]:
# col_dict

In [22]:
core = ['channelLink','channelTitle','currency','exchange',
        'timestamp','signalTime','pumptime_edited','coin_name']
col_dict['core'] = core

In [23]:
col_dict.keys()

dict_keys(['open', 'high', 'low', 'close', 'volumefrom', 'volumeto', 'core'])

In [24]:
ohlcv_vars = []
for k,v in col_dict.items():
    if k not in ['core','volumefrom']:
        ohlcv_vars+=col_dict[k]

## Make sure all the importmnat columns are preserved

In [25]:
len(ohlcv_vars)

160

In [26]:
orig_cols = x_final.columns
for i in orig_cols:
    if i not in ohlcv_vars:
        print(i)

NameError: name 'x_final' is not defined

In [58]:
x_final = x_final[ohlcv_vars]
x_final = x_final.reset_index(drop=True)

In [60]:
x_final.head(2)

,open_d0,open_d1,open_d2,open_d3,open_d4,open_d5,open_d6,open_d7,open_d8,open_d9,open_d10,open_d11,open_d12,open_d13,open_d14,open_d15,open_d16,open_d17,open_d18,open_d19,open_d20,open_d21,open_d22,open_d23,open_d24,open_d25,open_d26,open_d27,open_d28,open_d29,open_d30,open_d31,high_d0,high_d1,high_d2,high_d3,high_d4,high_d5,high_d6,high_d7,high_d8,high_d9,high_d10,high_d11,high_d12,high_d13,high_d14,high_d15,high_d16,high_d17,high_d18,high_d19,high_d20,high_d21,high_d22,high_d23,high_d24,high_d25,high_d26,high_d27,high_d28,high_d29,high_d30,high_d31,low_d0,low_d1,low_d2,low_d3,low_d4,low_d5,low_d6,low_d7,low_d8,low_d9,low_d10,low_d11,low_d12,low_d13,low_d14,low_d15,low_d16,low_d17,low_d18,low_d19,low_d20,low_d21,low_d22,low_d23,low_d24,low_d25,low_d26,low_d27,low_d28,low_d29,low_d30,low_d31,close_d0,close_d1,close_d2,close_d3,close_d4,close_d5,close_d6,close_d7,close_d8,close_d9,close_d10,close_d11,close_d12,close_d13,close_d14,close_d15,close_d16,close_d17,close_d18,close_d19,close_d20,close_d21,close_d22,close_d23,close_d24,close_d25,close_d26,close_d27,close_d28,close_d29,close_d30,close_d31,volumefrom_d0,volumefrom_d1,volumefrom_d2,volumefrom_d3,volumefrom_d4,volumefrom_d5,volumefrom_d6,volumefrom_d7,volumefrom_d8,volumefrom_d9,volumefrom_d10,volumefrom_d11,volumefrom_d12,volumefrom_d13,volumefrom_d14,volumefrom_d15,volumefrom_d16,volumefrom_d17,volumefrom_d18,volumefrom_d19,volumefrom_d20,volumefrom_d21,volumefrom_d22,volumefrom_d23,volumefrom_d24,volumefrom_d25,volumefrom_d26,volumefrom_d27,volumefrom_d28,volumefrom_d29,volumefrom_d30,volumefrom_d31,volumeto_d0,volumeto_d1,volumeto_d2,volumeto_d3,volumeto_d4,volumeto_d5,volumeto_d6,volumeto_d7,volumeto_d8,volumeto_d9,volumeto_d10,volumeto_d11,volumeto_d12,volumeto_d13,volumeto_d14,volumeto_d15,volumeto_d16,volumeto_d17,volumeto_d18,volumeto_d19,volumeto_d20,volumeto_d21,volumeto_d22,volumeto_d23,volumeto_d24,volumeto_d25,volumeto_d26,volumeto_d27,volumeto_d28,volumeto_d29,volumeto_d30,volumeto_d31,channelLink,channelTitle,currency,exchange,timestamp,signalTime,pumptime_edited,coin_name
0,294.0,294.0,294.0,294.0,298.0,298.0,298.0,323.0,323.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,504.0,504.0,504.0,504.0,504.0,378.0,378.0,586.0,586.0,586.0,586.0,586.0,586.0,1032.0,294.0,294.0,294.0,298.0,298.0,298.0,323.0,323.0,323.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,504.0,504.0,504.0,504.0,504.0,504.0,378.0,591.0,586.0,586.0,586.0,586.0,586.0,293.0,294.0,294.0,294.0,294.0,298.0,298.0,298.0,323.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,504.0,504.0,504.0,504.0,378.0,378.0,378.0,586.0,586.0,586.0,586.0,586.0,355.0,294.0,294.0,294.0,294.0,298.0,298.0,298.0,323.0,323.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,263.0,504.0,504.0,504.0,504.0,504.0,378.0,378.0,586.0,586.0,586.0,586.0,586.0,2.550788e+12,1.000000e+00,1.000000e+00,1.000000e+00,4.199000e+09,1.000000e+00,1.000000e+00,5.111000e+09,1.000000e+00,9.387000e+09,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.085750e+11,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,7.774000e+09,1.000000e+00,6.938300e+10,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.811000e+07,1.000000e+00,1.000000e+00,1.000000e+00,1.235000e+04,1.000000e+00,1.000000e+00,1.530000e+04,1.000000e+00,3.032000e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.575000e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.918000e+04,1.000000e+00,3.703000e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,https://t.me/tornadosignals,Tornado Signals,TEC,Yobit,1532115901,2018-07-20T19:45:01.689,2018-07-20 19:45:01+00:00,TEC
1,11040000.0,11250000.0,11680000.0,11950000.0,11400000.0,11170000.0,11200000.0,10990000.0,11100000.0,10900000.0,11010000.0,11170000.0,11380000.0,11130000.0,11340000.0,11590000.0,11770000.0,11850000.0

In [61]:
x_final.to_pickle("ohlcv_data_training_d.pkl")

### Also save the column names just to be sure


In [28]:
f = open("ohlcv_dict_d.pkl","wb")
pickle.dump(col_dict,f)
f.close()

g = open("ohlcv_features_d.pkl","wb")
pickle.dump(ohlcv_vars,g)
g.close()

In [27]:
ohlcv_vars

['open_d0',
 'open_d1',
 'open_d2',
 'open_d3',
 'open_d4',
 'open_d5',
 'open_d6',
 'open_d7',
 'open_d8',
 'open_d9',
 'open_d10',
 'open_d11',
 'open_d12',
 'open_d13',
 'open_d14',
 'open_d15',
 'open_d16',
 'open_d17',
 'open_d18',
 'open_d19',
 'open_d20',
 'open_d21',
 'open_d22',
 'open_d23',
 'open_d24',
 'open_d25',
 'open_d26',
 'open_d27',
 'open_d28',
 'open_d29',
 'open_d30',
 'open_d31',
 'high_d0',
 'high_d1',
 'high_d2',
 'high_d3',
 'high_d4',
 'high_d5',
 'high_d6',
 'high_d7',
 'high_d8',
 'high_d9',
 'high_d10',
 'high_d11',
 'high_d12',
 'high_d13',
 'high_d14',
 'high_d15',
 'high_d16',
 'high_d17',
 'high_d18',
 'high_d19',
 'high_d20',
 'high_d21',
 'high_d22',
 'high_d23',
 'high_d24',
 'high_d25',
 'high_d26',
 'high_d27',
 'high_d28',
 'high_d29',
 'high_d30',
 'high_d31',
 'low_d0',
 'low_d1',
 'low_d2',
 'low_d3',
 'low_d4',
 'low_d5',
 'low_d6',
 'low_d7',
 'low_d8',
 'low_d9',
 'low_d10',
 'low_d11',
 'low_d12',
 'low_d13',
 'low_d14',
 'low_d15',
 'low_